In [7]:
from models_train.IQAmodel import IQAModel
import os
import numpy as np
import random
import torch
from argparse import ArgumentParser
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms.functional import resize, to_tensor, normalize
import torch.nn.functional as F
from PIL import Image
import h5py
from tqdm import tqdm
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

In [19]:
def denorm(mmin, mmax, df):
    df['clear_val'] = df['clear_val'].transform(lambda x: x*(mmax-mmin)+mmin)
    df['attacked_val_eps=2'] = df['attacked_val_eps=2'].transform(lambda x: x*(mmax-mmin)+mmin)
    df['attacked_val_eps=4'] = df['attacked_val_eps=4'].transform(lambda x: x*(mmax-mmin)+mmin)
    df['attacked_val_eps=6'] = df['attacked_val_eps=6'].transform(lambda x: x*(mmax-mmin)+mmin)
    df['attacked_val_eps=8'] = df['attacked_val_eps=8'].transform(lambda x: x*(mmax-mmin)+mmin)
    df['attacked_val_eps=10'] = df['attacked_val_eps=10'].transform(lambda x: x*(mmax-mmin)+mmin)

In [20]:
path = "KonIQ-10k_resnet101+cayley_pool+0.1_l2+relu_PGD=1.csv"
CSV_PATH = "/home/28i_mel@lab.graphicon.ru/mnt/dione/28i_mel"
weights_path = "activation=relu-Linearity-resnet101-bs=8-loss=norm-in-norm-p=1.0-q=2.0-detach-False-KonIQ-10k-res=True-498x664-clp=True+prune=0.1l2_lr=1e-06_e=5_iters=1"

In [21]:
ckpt = torch.load(os.path.join(CSV_PATH, "Linearity-ckpt", weights_path))

/tmp/ipykernel_1900921/4219770622.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(os.path.join(CSV_PATH, "Linearity-ckpt", weights_path))


In [22]:
(ckpt['min'], ckpt['max'])

(np.float64(11.457558137907661), np.float64(85.37204118074094))

In [23]:
df = pd.read_csv(os.path.join(CSV_PATH, "arch_Linearity-results", path))

In [24]:
df

,Unnamed: 0,image_name,clear_val,attacked_val_eps=2,attacked_val_eps=4,attacked_val_eps=6,attacked_val_eps=8,attacked_val_eps=10
0,0,10007357496.jpg,0.719772,1.023124,0.960880,0.904714,0.825748,0.756405
1,1,10020766793.jpg,0.837871,1.090275,1.016461,0.932748,0.856933,0.778032
2,2,10020891105.jpg,0.460688,0.820885,0.790184,0.750841,0.710772,0.671122
3,3,10022757465.jpg,0.760048,0.898828,0.876023,0.837898,0.794696,0.757410
4,4,10039534103.jpg,0.736789,0.756664,0.709110,0.677831,0.648943,0.618422
...,...,...,...,...,...,...,...,...
2010,2010,9936084895.jpg,0.670260,0.822609,0.735966,0.674834,0.634832,0.590485
2011,2011,9977056676.jpg,0.415844,0.742931,0.700645,0.619917,0.593321,0.560026
2012,2012,9977191265.jpg,0.717124,0.880841,0.815992,0.743976,0.733537,0.698149
2013,2013,9984535544.jpg,0.753179,0.908474,0.854987,0.794753,0.761519,0.707371


In [25]:
denorm(ckpt['min'], ckpt['max'], df)

In [26]:
df

,Unnamed: 0,image_name,clear_val,attacked_val_eps=2,attacked_val_eps=4,attacked_val_eps=6,attacked_val_eps=8,attacked_val_eps=10
0,0,10007357496.jpg,64.659131,87.081214,82.480530,78.329011,72.492325,67.366868
1,1,10020766793.jpg,73.388356,92.044644,86.588774,80.401157,74.797329,68.965387
2,2,10020891105.jpg,45.509055,72.132816,69.863589,66.955596,63.993896,61.063181
3,3,10022757465.jpg,67.636134,77.893979,76.208332,73.390364,70.197095,67.441099
4,4,10039534103.jpg,65.916968,67.385981,63.871093,61.559077,59.423821,57.167867
...,...,...,...,...,...,...,...,...
2010,2010,9936084895.jpg,60.999467,72.260252,65.856120,61.337598,58.380873,55.102934
2011,2011,9977056676.jpg,42.194464,66.370884,63.245377,57.278433,55.312584,52.851610
2012,2012,9977191265.jpg,64.463389,76.564434,71.771201,66.448145,65.676575,63.060880
2013,2013,9984535544.jpg,67.128387,78.606927,74.653505,70.201323,67.744852,63.742492


In [27]:
path = "KonIQ-10k_resnet101+relu_PGD=1.csv"
CSV_PATH = "/home/28i_mel@lab.graphicon.ru/mnt/dione/28i_mel"
weights_path = "activation=relu-Linearity-resnet101-bs=8-loss=norm-in-norm-p=1.0-q=2.0-detach-False-KonIQ-10k-res=True-498x664"
ckpt = torch.load(os.path.join(CSV_PATH, "Linearity-ckpt", weights_path))
df = pd.read_csv(os.path.join(CSV_PATH, "arch_Linearity-results", path))

/tmp/ipykernel_1900921/1575917506.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(os.path.join(CSV_PATH, "Linearity-ckpt", weights_path))


In [28]:
denorm(ckpt['min'], ckpt['max'], df)
df

,Unnamed: 0,image_name,clear_val,attacked_val_eps=2,attacked_val_eps=4,attacked_val_eps=6,attacked_val_eps=8,attacked_val_eps=10
0,0,10007357496.jpg,70.315854,83.152554,81.672195,79.418690,76.775910,74.198777
1,1,10020766793.jpg,74.534388,85.261233,83.536690,81.095620,78.792733,76.068698
2,2,10020891105.jpg,50.595276,74.905774,74.925679,73.391256,72.026780,71.224599
3,3,10022757465.jpg,67.659917,76.111226,75.940307,75.120154,73.706802,72.754519
4,4,10039534103.jpg,62.029984,66.788884,63.392309,62.276158,60.852901,60.959321
...,...,...,...,...,...,...,...,...
2010,2010,9936084895.jpg,58.194785,73.525217,70.949466,68.942616,68.740939,67.258095
2011,2011,9977056676.jpg,45.765211,67.562833,66.128988,64.777604,64.132401,64.020844
2012,2012,9977191265.jpg,63.821209,76.877108,74.973585,74.288540,74.662715,74.520036
2013,2013,9984535544.jpg,68.005197,78.810692,77.497840,75.675555,74.998190,73.900776


In [29]:
path = "KonIQ-10k_resnet101+relu_PGD=1.csv"
CSV_PATH = "/home/28i_mel@lab.graphicon.ru/mnt/dione/28i_mel"
weights_path = "activation=relu-Linearity-resnet101-bs=8-loss=norm-in-norm-p=1.0-q=2.0-detach-False-KonIQ-10k-res=True-498x664-clp=True"
ckpt = torch.load(os.path.join(CSV_PATH, "Linearity-ckpt", weights_path))
df = pd.read_csv(os.path.join(CSV_PATH, "arch_Linearity-results", path))

/tmp/ipykernel_1900921/2109238995.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(os.path.join(CSV_PATH, "Linearity-ckpt", weights_path))


In [30]:
denorm(ckpt['min'], ckpt['max'], df)
df

,Unnamed: 0,image_name,clear_val,attacked_val_eps=2,attacked_val_eps=4,attacked_val_eps=6,attacked_val_eps=8,attacked_val_eps=10
0,0,10007357496.jpg,70.967725,83.384344,81.952430,79.772671,77.216376,74.723580
1,1,10020766793.jpg,75.048208,85.424016,83.755909,81.394723,79.167198,76.532307
2,2,10020891105.jpg,51.892503,75.407440,75.426694,73.942485,72.622661,71.846731
3,3,10022757465.jpg,68.398704,76.573443,76.408118,75.614804,74.247705,73.326585
4,4,10039534103.jpg,62.953011,67.556175,64.270753,63.191128,61.814448,61.917385
...,...,...,...,...,...,...,...,...
2010,2010,9936084895.jpg,59.243318,74.072062,71.580602,69.639427,69.444350,68.010031
2011,2011,9977056676.jpg,47.220503,68.304797,66.917875,65.610714,64.986626,64.878719
2012,2012,9977191265.jpg,64.685618,77.314262,75.473032,74.810405,75.172335,75.034325
2013,2013,9984535544.jpg,68.732684,79.184569,77.914680,76.152030,75.496832,74.435331


: 